<a href="https://colab.research.google.com/github/thesteve0/impatient-computer-vision/blob/main/7_visual_language_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visual Language Models

The models we have used up to now could only handle one mode of unstructured data - images. These next type of models we are going to cover are called multi-modal, they can handle multiple modes of input. In this case, the models are trained on both images and text. The training of these models occurs with image and text pairs. The model captures and combines the features, embeddings, allowing them to generate predictions for either modality.

What this practically means is you can query the images using text, such as "images with cats". And you can also ask the model to describe what is in the image. You can still do the single mode tasks, such as object detection, and you gain the benefit of the model understanding more of the human perceptions of the image.

One of the earliest models of this type was built by Open AI and called CLIP. You actually saw the embeddings generated from this model back in the embeddings notebook. Those embeddings were generated using an open weights version of CLIP called OpenClip. And because these embeddings were generated using this multi-modal model, they contained more of the semantic features, human understanding, of what was in the images.

We are going to use another YOLO model to help with one of the most labor-intensive part of training computer vision models, creating annotation data. This use is sometimes called generative labeling. We use the models to generate an initial set of labels. While we still have  edit the labels, we avoid trying to do everything from scratch.

And with that, time to get our hands dirty - let's do the housekeeping:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install fiftyone==1.9.0 torch torchvision ultralytics transformers git+https://github.com/ultralytics/CLIP.git qwen-vl-utils accelerate

import fiftyone as fo
import fiftyone.zoo as foz

name = "our-photos"
dir = "/content/drive/MyDrive/impatient-cv/flickr-labeled"

dataset = fo.Dataset.from_dir(
    dataset_dir=dir,
    dataset_type=fo.types.FiftyOneDataset,
    name=name
)


## Running models

There are all sorts of interesting tasks you can do with these multi-modal models, for example, [this code](https://github.com/thesteve0/photo-explorer/blob/main/4_word_cloud.py) generates a word cloud from ideas in the dataset's images. Today we are going to use these models to help us generate annotations in a dataset without annotations.

When we did our original object detections, the model could only produce predictions from the classes it was trained on. We can use the multi-modal models to generate custom object detection classes. Generating these labels would be one of the first steps in training or fine-tuning our own model to work for our use cases.

In [ ]:
# Just default predictions
model = foz.load_zoo_model("yolov8x-world-torch")

dataset.apply_model(model, label_field="yolo_default_predictions", num_workers=4, batch_size=2)

# #
# # Make zero-shot predictions with custom classes
# #
# print("Now let's do the custom classes")


# model = foz.load_zoo_model(
#     "yolov8x-world-torch",
#     classes=["car", "motocycle", "food", "flower", "animals", "tree"],
# )

# dataset.apply_model(model, label_field="yolo_class_predictions", num_workers=4, batch_size=2)

######## OMDET - https://docs.voxel51.com/model_zoo/models/omdet_turbo_swin_tiny_torch.html
model = foz.load_zoo_model(
    "omdet-turbo-swin-tiny-torch",
    classes=["car", "motocycle", "food", "flower", "animals", "tree"],
)

dataset.apply_model(model, label_field="omdet_class_predictions", num_workers=10, batch_size=6, confidence_thresh=0.1)

######## Owl vit patch 16 - https://docs.voxel51.com/model_zoo/models/owlvit_base_patch16_torch.html
model = foz.load_zoo_model(
    "owlvit-base-patch16-torch",
    classes=["car", "motocycle", "food", "flower", "animals", "tree"],
)

dataset.apply_model(model, label_field="owl_class_predictions", num_workers=10, batch_size=6, confidence_thresh=0.01)

######## Paligemma - Gated
# foz.register_zoo_model_source("https://github.com/harpreetsahota204/paligemma2")
# foz.download_zoo_model(
#     "https://github.com/harpreetsahota204/paligemma2",
#     model_name="google/paligemma2-3b-mix-224",
# )
# model = foz.load_zoo_model(
#     "google/paligemma2-3b-mix-224"
#     )
# model.operation = "detection"
# model.prompt = ["car", "motocycle", "food", "flower", "animals", "tree"]
# dataset.apply_model(model, label_field="Paligemma_detections", num_workers=10, batch_size=4, confidence_thresh=0.00001)


######## Qwen 2,5 VL - 7.5 gig download - needs something else to work
# foz.register_zoo_model_source("https://github.com/harpreetsahota204/qwen2_5_vl")
# foz.download_zoo_model(
#     "https://github.com/harpreetsahota204/qwen2_5_vl",
#     model_name="Qwen/Qwen2.5-VL-3B-Instruct",

# )
# model = foz.load_zoo_model(
#     "Qwen/Qwen2.5-VL-3B-Instruct",
#     )
# model.operation = "detect"
# model.prompt = ["car", "motocycle", "food", "flower", "animals", "tree"]
# dataset.apply_model(model, label_field="qwen_class_predictions", batch_size=6, confidence_thresh=0.01)


######## MiniCPM-V model source - 16 gig download takes too long
# foz.register_zoo_model_source(
#     "https://github.com/harpreetsahota204/minicpm-v",
#     overwrite=True
# )

# model = foz.load_zoo_model(
#     "openbmb/MiniCPM-V-4_5"
# )
# model.operation = "detect"
# model.prompt = ["car", "motocycle", "food", "flower", "animals", "tree"]
# dataset.apply_model(model, label_field="MiniCPM_detections", num_workers=10, batch_size=4, confidence_thresh=0.00001)

session = fo.launch_app(dataset, auto=False)
session.url

## Summary

You can why these techniques are extremely powerful and I hope they help you as you start to create your own vision datasets and models. These datasets will actually be crucial for the final notebook on fine tuning models. Remember, we will only discuss this if there is time but you can use it to help you approach this task in the future

[Fine Tuning](https://github.com/thesteve0/impatient-computer-vision/blob/main/8_fine_tuning.ipynb)